In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url_gafas_sol = "https://www.millerandmarc.com/collections/sol"

dicc = {"nombre": [],
        "precio": [],
        "color": []}
lista_colores = ["black", "brown", "grey"]
try:
    for color in tqdm(lista_colores):
        driver = webdriver.Chrome()
        driver.get(url_gafas_sol)
        driver.implicitly_wait(10)
        driver.maximize_window()
        driver.execute_script("window.scrollBy(0, 600);")
        sleep(0.5)
        if color == "black":
            driver.find_element(By.XPATH, "//*[@id='gf-tree']/div[3]/div[4]/div[2]/div/ul/li[1]/button").click()
        elif color == "brown":
            driver.find_element(By.XPATH, "//*[@id='gf-tree']/div[3]/div[4]/div[2]/div/ul/li[3]/button").click()
        elif color == "grey":
            driver.find_element(By.XPATH, "//*[@id='gf-tree']/div[3]/div[4]/div[2]/div/ul/li[7]/button").click()
        sleep(0.5)
        contenedor_productos = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='gf-products']"))
        )
        sleep(0.5)
        productos = contenedor_productos.find_elements(By.XPATH, ".//a[contains(@class, 'spf-product-card__image-wrapper') and contains(@class, 'hover-effect')]")
        urls_productos = [producto.get_attribute("href") for producto in productos]
        sleep(0.5)
        for url in urls_productos:
            driver.get(url)
            sleep(0.5)
            nombre = driver.find_element(By.XPATH, "//*[@id='xtitle']/h1").text
            precio = driver.find_element(By.XPATH, "//*[@id='price-template--23785914040706__main']/div/div/div[1]/span[2]").text
            color_gafas = driver.find_element(By.XPATH, "//*[@id='MainProduct-template--23785914040706__main']/x-grid/x-cell[3]/product-content/section[4]/div/div/div[2]/div/legend/span").text
            dicc["nombre"].append(nombre)
            dicc["precio"].append(precio)
            dicc["color"].append(color_gafas)
        driver.close()
except:
    print("Error al encontrar el archivo")


In [ ]:
df = pd.DataFrame(dicc)
df["precio"] = df["precio"].str.replace("€", "").astype(float)
df

In [ ]:
df.to_csv("../datos/df_millerandmarc.csv")